In [120]:
import pandas as pd

In [121]:
df = pd.read_excel(r"references\Subject_info.xlsx")

In [122]:
# Assuming df is your DataFrame
def preprocess_labels(df):
    # Mapping similar categories to a single category
    label_mapping = {
        'EMCI': 'MCI',
        'LMCI': 'MCI',
        'SMC': 'CN'  # If you want SMC to be considered as CN, include this; remove if not
    }
    df['Research Group'] = df['Research Group'].replace(label_mapping)
    return df

preprocess_labels(df)

,Subject,Sex,Weight,Research Group,APOE A1,APOE A2,Age,dataset_split,File_Path,File_Path_desktop,PATH
0,002_S_0295,M,73.0,CN,3.0,4.0,90.0,train,C:\Users\Micha\OneDrive - Høyskolen Kristiania...,D:\Data\Preprocessed\Fused Images\002_S_0295_f...,data\processed\002_S_0295_fused.nii
1,002_S_0413,F,57.6,CN,3.0,3.0,81.5,test,C:\Users\Micha\OneDrive - Høyskolen Kristiania...,D:\Data\Preprocessed\Fused Images\002_S_0413_f...,data\processed\002_S_0413_fused.nii
2,002_S_0685,F,68.9,CN,3.0,3.0,95.8,test,C:\Users\Micha\OneDrive - Høyskolen Kristiania...,D:\Data\Preprocessed\Fused Images\002_S_0685_f...,data\processed\002_S_0685_fused.nii
3,002_S_0729,F,65.8,MCI,3.0,4.0,71.3,validation,C:\Users\Micha\OneDrive - Høyskolen Kristiania...,D:\Data\Preprocessed\Fused Images\002_S_0729_f...,data\processed\002_S_0729_fused.nii
4,002_S_1155,M,64.9,MCI,3.0,3.0,64.0,train,C:\Users\Micha\OneDrive - Høyskolen Kristiania...,D:\Data\Preprocessed\Fused Images\002_S_1155_f...,data\processed\002_S_1155_fused.nii
...,...,...,...,...,...,...,...,...,...,...,...
173,941_S_4377,F,121.6,MCI,3.0,4.0,69.5,test,C:\Users\Micha\OneDrive - Høyskolen Kristiania...,D:\Data\Preprocessed\Fused Images\941_S_4377_f...,data\processed\941_S_4377_fused.nii
174,941_S_4420,M,79.4,MCI,3.0,3.0,81.5,train,C:\Users\Micha\OneDrive - Høyskolen Kristiania...,D:\Data\Preprocessed\Fused Images\941_S_4420_f...,data\processed\941_S_4420_fused.nii
175,941_S_4764,F,77.6,MCI,3.0,3.0,82.8,train,C:\Users\Micha\OneDrive - Høyskolen Kristiania...,D:\Data\Preprocessed\Fused Images\941_S_4764_f...,data\processed\941_S_4764_fused.nii
176,941_S_5124,F,78.9,CN,3.0,3.0,76.8,test,C:\Users\Micha\OneDrive - Høyskolen Kristiania...,D:\Data\Preprocessed\Fused Images\941_S_5124_f...,data\processed\941_S_5124_fused.nii


In [123]:
import pandas as pd

# Assuming df is your DataFrame
df = df[df['Research Group'] != 'AD']


In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167 entries, 0 to 177
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Subject            167 non-null    object 
 1   Sex                167 non-null    object 
 2   Weight             167 non-null    float64
 3   Research Group     167 non-null    object 
 4   APOE A1            165 non-null    float64
 5   APOE A2            165 non-null    float64
 6   Age                167 non-null    float64
 7   dataset_split      167 non-null    object 
 8   File_Path          167 non-null    object 
 9   File_Path_desktop  167 non-null    object 
 10  PATH               167 non-null    object 
dtypes: float64(4), object(7)
memory usage: 15.7+ KB


In [125]:
df

,Subject,Sex,Weight,Research Group,APOE A1,APOE A2,Age,dataset_split,File_Path,File_Path_desktop,PATH
0,002_S_0295,M,73.0,CN,3.0,4.0,90.0,train,C:\Users\Micha\OneDrive - Høyskolen Kristiania...,D:\Data\Preprocessed\Fused Images\002_S_0295_f...,data\processed\002_S_0295_fused.nii
1,002_S_0413,F,57.6,CN,3.0,3.0,81.5,test,C:\Users\Micha\OneDrive - Høyskolen Kristiania...,D:\Data\Preprocessed\Fused Images\002_S_0413_f...,data\processed\002_S_0413_fused.nii
2,002_S_0685,F,68.9,CN,3.0,3.0,95.8,test,C:\Users\Micha\OneDrive - Høyskolen Kristiania...,D:\Data\Preprocessed\Fused Images\002_S_0685_f...,data\processed\002_S_0685_fused.nii
3,002_S_0729,F,65.8,MCI,3.0,4.0,71.3,validation,C:\Users\Micha\OneDrive - Høyskolen Kristiania...,D:\Data\Preprocessed\Fused Images\002_S_0729_f...,data\processed\002_S_0729_fused.nii
4,002_S_1155,M,64.9,MCI,3.0,3.0,64.0,train,C:\Users\Micha\OneDrive - Høyskolen Kristiania...,D:\Data\Preprocessed\Fused Images\002_S_1155_f...,data\processed\002_S_1155_fused.nii
...,...,...,...,...,...,...,...,...,...,...,...
173,941_S_4377,F,121.6,MCI,3.0,4.0,69.5,test,C:\Users\Micha\OneDrive - Høyskolen Kristiania...,D:\Data\Preprocessed\Fused Images\941_S_4377_f...,data\processed\941_S_4377_fused.nii
174,941_S_4420,M,79.4,MCI,3.0,3.0,81.5,train,C:\Users\Micha\OneDrive - Høyskolen Kristiania...,D:\Data\Preprocessed\Fused Images\941_S_4420_f...,data\processed\941_S_4420_fused.nii
175,941_S_4764,F,77.6,MCI,3.0,3.0,82.8,train,C:\Users\Micha\OneDrive - Høyskolen Kristiania...,D:\Data\Preprocessed\Fused Images\941_S_4764_f...,data\processed\941_S_4764_fused.nii
176,941_S_5124,F,78.9,CN,3.0,3.0,76.8,test,C:\Users\Micha\OneDrive - Høyskolen Kristiania...,D:\Data\Preprocessed\Fused Images\941_S_5124_f...,data\processed\941_S_5124_fused.nii


In [126]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import nibabel as nib
import pandas as pd
from torchvision import transforms
from tqdm import tqdm


In [127]:
class NiiDataset(Dataset):
    def __init__(self, df, transform=None):
        """
        Initializes the dataset object.
        :param df: DataFrame containing file paths, labels, and subject IDs.
        :param transform: A function or a series of transforms to apply to the images.
        """
        self.paths = df['PATH'].tolist()  # Paths to .nii files
        # Convert labels to categorical codes and maintain a mapping
        self.label_mapping = {category: code for code, category in enumerate(pd.Categorical(df['Research Group']).categories)}
        self.labels = pd.Categorical(df['Research Group'], categories=self.label_mapping.keys()).codes
        self.subjects = df['Subject'].tolist()  # Subject identifiers
        self.transform = transform  # Transformation function(s)

    def __len__(self):
        """
        Returns the total number of samples in the dataset.
        """
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        try:
            image = nib.load(path).get_fdata()
            image = np.expand_dims(image, axis=0)  # Add a channel dimension
            image = image.astype(np.float32)  # Ensure the image is float32
            if self.transform:
                image = self.transform(image)
        except Exception as e:
            print(f"Error loading NII file at {path}: {e}")
            image = np.zeros((1, 64, 64, 64), dtype=np.float32)  # Fallback image of the same type

        label = torch.tensor(self.labels[idx], dtype=torch.long)
        subject = self.subjects[idx]

        return image, label, path, subject



In [128]:
# Load datasets
def load_datasets(df):
    train_df = df[df['dataset_split'] == 'train']
    val_df = df[df['dataset_split'] == 'validation']
    test_df = df[df['dataset_split'] == 'test']
    
    train_dataset = NiiDataset(train_df)
    val_dataset = NiiDataset(val_df)
    test_dataset = NiiDataset(test_df)
    
    return train_dataset, val_dataset, test_dataset


In [129]:
def create_dataloaders(train_dataset, val_dataset, test_dataset, batch_size=4):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    return train_loader, val_loader, test_loader


In [130]:
# Model definition
class Baseline3DCNN(nn.Module):
    def __init__(self, num_classes=3, init_filters=32, kernel_size=3, stride=2, num_fc_units=128):
        super(Baseline3DCNN, self).__init__()
        self.conv1 = nn.Conv3d(1, init_filters, kernel_size=kernel_size, stride=stride, padding=1)
        self.conv2 = nn.Conv3d(init_filters, init_filters*2, kernel_size=kernel_size, stride=stride, padding=1)
        self.conv3 = nn.Conv3d(init_filters*2, init_filters*4, kernel_size=kernel_size, stride=stride, padding=1)
        self.pool = nn.MaxPool3d(2)
        self.relu = nn.ReLU()

        # Compute the flattened size after all convolutions and pooling
        self.final_dim = self._get_conv_output_dim(193, 3, stride, kernel_size, init_filters*4)
        self.fc1 = nn.Linear(self.final_dim, num_fc_units)
        self.fc2 = nn.Linear(num_fc_units, num_classes)

    def _get_conv_output_dim(self, input_dim, num_convs, stride, kernel_size, num_filters):
        output_dim = input_dim
        for _ in range(num_convs):
            output_dim = ((output_dim - kernel_size + 2 * (kernel_size // 2)) // stride + 1) // 2  # Pooling divides size by 2
        return output_dim * output_dim * output_dim * num_filters

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [131]:
# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


In [132]:
# Assuming 'df' is your DataFrame loaded with the 'Research Group' column available
label_categories = pd.Categorical(df['Research Group'])
label_mapping = {code: category for code, category in enumerate(label_categories.categories)}


In [133]:
def train_model(model, train_loader, criterion, optimizer, label_mapping, num_epochs=10, device='cpu'):
    model.to(device)
    model.train()
    train_results = []
    for epoch in range(num_epochs):
        for images, labels, paths, subjects in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, predicted_indices = torch.max(outputs, 1)
            predicted_labels = [label_mapping[code] for code in predicted_indices.cpu().numpy()]

            for label, pred, path, subject in zip(labels.cpu().numpy(), predicted_labels, paths, subjects):
                train_results.append({
                    'Subject': subject,
                    'Path': path,
                    'Actual Label': label_mapping[label.item()],
                    'Prediction': pred,
                    'Type': 'Train'
                })
    return train_results

def validate_model(model, val_loader, criterion, label_mapping, device='cpu'):
    model.to(device)
    model.eval()
    validation_results = []
    with torch.no_grad():
        for images, labels, paths, subjects in tqdm(val_loader, desc='Validation'):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted_indices = torch.max(outputs, 1)
            predicted_labels = [label_mapping[code] for code in predicted_indices.cpu().numpy()]

            for label, pred, path, subject in zip(labels.cpu().numpy(), predicted_labels, paths, subjects):
                validation_results.append({
                    'Subject': subject,
                    'Path': path,
                    'Actual Label': label_mapping[label.item()],
                    'Prediction': pred,
                    'Type': 'Validation'
                })
    return validation_results

def test_model(model, test_loader, label_mapping, device='cpu'):
    model.to(device)
    model.eval()
    test_results = []
    with torch.no_grad():
        for images, labels, paths, subjects in tqdm(test_loader, desc='Testing'):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted_indices = torch.max(outputs, 1)
            predicted_labels = [label_mapping[code] for code in predicted_indices.cpu().numpy()]

            for label, pred, path, subject in zip(labels.cpu().numpy(), predicted_labels, paths, subjects):
                test_results.append({
                    'Subject': subject,
                    'Path': path,
                    'Actual Label': label_mapping[label.item()],
                    'Prediction': pred,
                    'Type': 'Test'
                })
    return test_results


In [134]:
train_dataset, val_dataset, test_dataset = load_datasets(df)  # Create datasets
train_loader, val_loader, test_loader = create_dataloaders(train_dataset, val_dataset, test_dataset, batch_size=4)  # Create dataloaders

In [135]:
# Configuration
config = {
    'num_classes': 3,
    'init_filters': 32,
    'kernel_size': 3,
    'stride': 2,
    'num_fc_units': 128,
    'optimizer': 'Adam',
    'loss_criterion': 'CrossEntropyLoss',
    'num_epochs': 10
}
config_df = pd.DataFrame([config])

# Model Initialization
model = Baseline3DCNN(num_classes=config['num_classes'],
                      init_filters=config['init_filters'],
                      kernel_size=config['kernel_size'],
                      stride=config['stride'],
                      num_fc_units=config['num_fc_units']).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Prepare DataLoaders (assuming your datasets are already split and loaded into variables train_dataset, val_dataset, test_dataset)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Training, Validation, Testing
train_results = train_model(model, train_loader, criterion, optimizer, label_mapping, num_epochs=config['num_epochs'], device=device)
validate_results = validate_model(model, val_loader, criterion, label_mapping, device=device)
test_results = test_model(model, test_loader, label_mapping, device=device)

# Combine results from all phases
all_results = pd.DataFrame(train_results + validate_results + test_results)

import datetime

# Assuming config_df and all_results are defined elsewhere in your script
# Get the current date and time
current_time = datetime.datetime.now()
formatted_time = current_time.strftime('%Y-%m-%d_%H-%M-%S')  # Format as Year-Month-Day_Hour-Minute-Second

# Create a unique filename for each experiment run
filename = f'{formatted_time}_Experiment.xlsx'

# Export to Excel with a unique filename based on the current date and time
with pd.ExcelWriter(filename) as writer:
    config_df.to_excel(writer, sheet_name='Configuration')
    all_results.to_excel(writer, sheet_name='Results')


Epoch 1/10:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 1/10:  32%|███▏      | 8/25 [00:20<00:42,  2.49s/it]

In [ ]:
try:
    image = nib.load(path).get_fdata()  # Load the image data
except Exception as e:
    print(f"Error loading NII file at {path}: {e}")
    # Handle the error, e.g., by skipping this file or using a default image


NameError: name 'path' is not defined